# Import data

In [16]:
%matplotlib inline

import glob
import os
import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras import optimizers
from keras import applications
from keras.utils.np_utils import to_categorical 
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# Dimensions of our images.
img_width, img_height = 240, 240

n_classes = 6451
train_data_dir = 'data/train'
valid_data_dir = 'data/valid'

In [21]:
epochs = 100
batch_size = 32

train_samples = 859486
valid_samples = 294058
#train_samples = 8594
#valid_samples = 2940

In [22]:
# train_datagen = ImageDataGenerator(
#         rescale=1. / 255,      # Normalize pixel values to [0,1]
#         shear_range=0.2,       # Randomly applies shearing transformation
#         zoom_range=0.2,        # Randomly applies shearing transformation
#         horizontal_flip=True)  # Randomly flip the images

train_datagen = ImageDataGenerator(rescale=1./255)

In [23]:
# train_generator = train_datagen.flow_from_directory(
#         train_data_dir,
#         target_size=(img_width, img_height),
#         batch_size=batch_size)

train_generator_bottleneck = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

Found 859486 images belonging to 6451 classes.


In [24]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [25]:
# valid_generator = datagen.flow_from_directory(
#         valid_data_dir,
#         target_size=(img_width, img_height),
#         batch_size=batch_size)

valid_generator_bottleneck = datagen.flow_from_directory(
        valid_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

Found 294058 images belonging to 6451 classes.


# Loading VGG 16 models

In [10]:
model_vgg = applications.VGG16(include_top=False, weights='imagenet')

58900480/58889256 [==============================] - 17s 0us/step


In [ ]:
bottleneck_features_train = model_vgg.predict_generator(train_generator_bottleneck, train_samples // batch_size)
np.save(open('models/bottleneck_features_train.npy', 'wb'), bottleneck_features_train)

In [ ]:
bottleneck_features_validation = model_vgg.predict_generator(valid_generator_bottleneck, valid_samples // batch_size)
np.save(open('models/bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

In [ ]:
num_classes = len(train_generator_bottleneck.class_indices)

train_data = np.load(open('models/bottleneck_features_train.npy', 'rb'))
train_labels = to_categorical(train_generator_bottleneck.classes[:859486], num_classes=num_classes)

validation_data = np.load(open('models/bottleneck_features_validation.npy', 'rb'))
validation_labels = to_categorical(valid_generator_bottleneck.classes[:294058], num_classes=num_classes)

In [ ]:
model_top = Sequential()
model_top.add(Flatten(input_shape=train_data.shape[1:]))
model_top.add(Dense(512, activation='relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(n_classes, activation='softmax'))

model_top.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

# Build Model

In [29]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation('softmax'))

In [30]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training

In [ ]:
# Model saving callback
# checkpointer = ModelCheckpoint(filepath='models/data_augmentation.h5', monitor='val_acc', verbose=1, save_best_only=True)
checkpointer = ModelCheckpoint(filepath='models/bottleneck_features.h5', monitor='val_acc', verbose=1, save_best_only=True)

# Early stopping
early_stopping = EarlyStopping(monitor='val_acc', verbose=1, patience=5)

# history = model.fit_generator(
#         train_generator,
#         steps_per_epoch=train_samples // batch_size,
#         epochs=epochs,
#         callbacks=[checkpointer, early_stopping],
#         verbose=2,
#         validation_data=valid_generator,
#         validation_steps=validation_samples // batch_size,)

history = model_top.fit(
        train_data,
        train_labels,
        verbose=2,
        epochs=epochs, 
        batch_size=batch_size,
        callbacks=[checkpointer, early_stopping],
        validation_data=(validation_data, validation_labels))

# Evaluation

In [ ]:
model.evaluate_generator(valid_generator, valid_samples)

In [ ]:
fig, ax = plt.subplots()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
#fig.savefig('../images/data_augmentation.svg', format='svg', dpi=1200)

In [1]:
a,b,c = [], [], []

In [2]:
a.append('sdjkf')

In [3]:
if a or b  or c:
    print a

['sdjkf']


In [4]:
a = []

In [5]:
if a or b  or c:
    print a